In [1]:
temp_pdf_path = 'manual.pdf'

In [2]:
!wget https://www.cryptopro.ru/sites/default/files/products/pdf/files/CryptoProPDF_UserGuide.pdf -O {temp_pdf_path}

--2024-12-10 11:31:20--  https://www.cryptopro.ru/sites/default/files/products/pdf/files/CryptoProPDF_UserGuide.pdf
193.37.157.43.cryptopro.ru (www.cryptopro.ru)... 
Connecting to www.cryptopro.ru (www.cryptopro.ru)|193.37.157.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1184743 (1.1M) [application/pdf]
Saving to: ‘manual.pdf’

manual.pdf          100%[===================>]   1.13M   973KB/s    in 1.2s    

2024-12-10 11:31:21 (973 KB/s) - ‘manual.pdf’ saved [1184743/1184743]



# Dependencies

In [ ]:
#!apt install -y poppler-utils tesseract-ocr tesseract-ocr-rus

In [3]:
!pip install PyPDF2 pdfminer.six pdfplumber pdf2image Pillow pytesseract

In [4]:
# Для считывания PDF
import PyPDF2
# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# Для извлечения текста из таблиц в PDF
import pdfplumber
# Для извлечения изображений из PDF
from PIL import Image
from pdf2image import convert_from_path
# Для выполнения OCR, чтобы извлекать тексты из изображений 
import pytesseract 
# Для удаления дополнительно созданных файлов
import os

# Layout extraction

In [5]:
pages = list(enumerate(extract_pages(temp_pdf_path)))

In [6]:
# Визуализация
# LTFigure, обозначающий область PDF, которая может быть изображениями, встроенными в страницу как другой документ PDF.
# LTTextContainer, обозначающий группу текстовых строк в прямоугольной области, которая затем анализируется, превращаясь в список объектов LTTextLine. Каждый из них представляет собой список объектов LTChar, в которых хранятся отдельные символы текста вместе с их метаданными.
# LTRect, обозначающий двухмерный прямоугольник, который можно использовать для задания рамок изображений или для создания таблиц в виде объекта LTPage.
for page_num, page in pages:
    print("Page num:", page_num)
    for elem in page:
        print(type(elem))

Page num: 0
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTRect'>
<class 'pdfminer.layout.LTFigure'>
<class 'pdfminer.layout.LTRect'>
<class 'pdfminer.layout.LTTextLineHorizontal'>
<class 'pdfminer.layout.LTTextLineHorizontal'>
Page num: 1
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>
<class 'pdfminer.layout.LTTextBoxHorizontal'>

In [7]:
def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()
    
    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))
    
    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)

# Text extraction (text layer)

In [8]:
def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()
    
    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))
    
    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)

In [10]:
# Визуализация первого попавшегося текстового блока
for page_num, page in pages:
    for elem in page:
        if isinstance(elem, LTTextContainer):
            break
    else:
        continue  # only executed if the inner loop did NOT break
    break  # only executed if the inner loop DID break
            
print("page num", page_num)
text_extraction(elem)

page num 0


('УТВЕРЖДЁН \n', [9.960000000000036, 'ABCDEE+Verdana'])

# Text extraction (image)

In [27]:
# Создаём функцию для вырезания элементов изображений из PDF
def crop_image(element, pageObj):
    # Получаем координаты для вырезания изображения из PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    # Обрезаем страницу по координатам (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Сохраняем обрезанную страницу в новый PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Сохраняем обрезанный PDF в новый файл
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Создаём функцию для преобразования PDF в изображения
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

# Создаём функцию для считывания текста из изображений
def image_to_text(image_path):
    # Считываем изображение
    img = Image.open(image_path)
    # Извлекаем текст из изображения
    text = pytesseract.image_to_string(img, lang="rus+eng")
    return text

def text_extraction_from_img(element, pageObj):
    crop_image(element, pageObj)
    convert_to_images("cropped_image.pdf")
    return image_to_text("PDF_image.png")

In [29]:
# Визуализация первой попавшейся картинки
pdfFileObj = open(temp_pdf_path, 'rb')
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

for page_num, page in pages:    
    for elem in page:
        if isinstance(elem, LTFigure):
            pageObj = pdfReaded.pages[page_num]
            result = text_extraction_from_img(elem, pageObj)
            if result.strip() == "":
                continue
            else:            
                print("page num", page_num)
                print(result)
                pdfFileObj.close()  
                break
    else:
        continue  # only executed if the inner loop did NOT break
    break  # only executed if the inner loop DID break


page num 4
ey КриптоПро PDF

Bac приветствует программа InstallShield Wizard
для КриптоПро PDF

Программа InstallShield(R) Wizard выполнит установку
КриптоПро PDF 1.6.0273 на компьютер. Для продолжения
нажмите кнопку "Далее".

ПРЕДУПРЕЖДЕНИЕ: Данная программа защищена законами
об авторских правах и международными соглашениями.

CryptoPro
РОВ




# Table extraction

In [30]:
def extract_table(pdf_path, page_num, table_num):
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    table_page = pdf.pages[page_num]
    # Извлекаем соответствующую таблицу
    table = table_page.extract_tables()[table_num]
    return table

# Преобразуем таблицу в соответствующий формат
def table_converter(table):
    table_string = ''
    # Итеративно обходим каждую строку в таблице
    for row_num in range(len(table)):
        row = table[row_num]
        # Удаляем разрыв строки из текста с переносом
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Преобразуем таблицу в строку
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Удаляем последний разрыв строки
    table_string = table_string[:-1]
    return table_string

In [35]:
# Визуализация первой попавшейся таблицы
for page_num, page in pages:    
    for elem in page:
        if isinstance(elem, LTRect):
            try:
                table = extract_table(temp_pdf_path, page_num, table_num=0)
                print(table_converter(table))
                break
            except:
                print("unable to extract table on page", page_num)
    else:
        continue  # only executed if the inner loop did NOT break
    break  # only executed if the inner loop DID break


unable to extract table on page 0
unable to extract table on page 0
unable to extract table on page 1
unable to extract table on page 1
unable to extract table on page 1
unable to extract table on page 2
unable to extract table on page 2
unable to extract table on page 3
unable to extract table on page 3
unable to extract table on page 3
unable to extract table on page 4
unable to extract table on page 4
unable to extract table on page 5
unable to extract table on page 5
unable to extract table on page 6
unable to extract table on page 6
unable to extract table on page 7
unable to extract table on page 7
unable to extract table on page 8
unable to extract table on page 8
unable to extract table on page 9
unable to extract table on page 9
unable to extract table on page 9
unable to extract table on page 10
unable to extract table on page 10
unable to extract table on page 10
unable to extract table on page 11
unable to extract table on page 11
unable to extract table on page 12
unable t